# ResNet-50 Training Script for Brain Tumor MRI Dataset (Multiclass Classification)
This backbone will be used in the Interpretable class wrapper for direct comparison with a ProtoPNet with a ResNet-50 backbone

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
import os

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            all_labels = []
            all_preds = []

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'test':
                precision = precision_score(all_labels, all_preds, average='weighted')
                recall = recall_score(all_labels, all_preds, average='weighted')
                f1 = f1_score(all_labels, all_preds, average='weighted')
                print(f'{phase} Precision: {precision:.4f} Recall: {recall:.4f} F1: {f1:.4f}')

                # deep copy the model
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = model.state_dict()

    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'Brain_Tumor_MRI.pth')
    return model

In [2]:
def main():
    data_dir = '/home/alan/Documents/YOLOV8_interpretable/Brain_Tumor_data_Mendeley'

    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
    dataloaders = {
        'train': DataLoader(image_datasets['train'], batch_size=16, shuffle=True, num_workers=4),  # Shuffle set to True
        'test': DataLoader(image_datasets['test'], batch_size=16, shuffle=False, num_workers=4)  # No need to shuffle validation set
    }

    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, len(image_datasets['train'].classes))

    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

    model_ft = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=25)

if __name__ == '__main__':
    main()

/home/alan/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/alan/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/24
----------
train Loss: 0.4684 Acc: 0.8186
test Loss: 0.2008 Acc: 0.9291
test Precision: 0.9361 Recall: 0.9291 F1: 0.9304
Epoch 1/24
----------
train Loss: 0.2606 Acc: 0.9037
test Loss: 0.1648 Acc: 0.9321
test Precision: 0.9408 Recall: 0.9321 F1: 0.9335
Epoch 2/24
----------
train Loss: 0.2031 Acc: 0.9286
test Loss: 0.0719 Acc: 0.9764
test Precision: 0.9765 Recall: 0.9764 F1: 0.9762
Epoch 3/24
----------
train Loss: 0.1644 Acc: 0.9419
test Loss: 0.0954 Acc: 0.9703
test Precision: 0.9716 Recall: 0.9703 F1: 0.9704
Epoch 4/24
----------
train Loss: 0.1547 Acc: 0.9442
test Loss: 0.0675 Acc: 0.9779
test Precision: 0.9790 Recall: 0.9779 F1: 0.9780
Epoch 5/24
----------
train Loss: 0.1267 Acc: 0.9561
test Loss: 0.0421 Acc: 0.9901
test Precision: 0.9902 Recall: 0.9901 F1: 0.9901
Epoch 6/24
----------
train Loss: 0.1244 Acc: 0.9552
test Loss: 0.0548 Acc: 0.9863
test Precision: 0.9864 Recall: 0.9863 F1: 0.9863
Epoch 7/24
----------
train Loss: 0.1108 Acc: 0.9606
test Loss: 0.0270 Acc: 

In [2]:
import torch.nn as nn
import torchvision.models as models

model = models.resnet50(pretrained=True)

def get_all_layers(module):
    layers = []
    for child in module.children():
        if isinstance(child, nn.Module):
            layers.append(child)
            layers.extend(get_all_layers(child))
    return layers

# Get a flat list of all layers
all_layers = get_all_layers(model)

# Print all layers with their indices
for idx, layer in enumerate(all_layers):
    print(f"Layer {idx}: {layer}")

# Identify the index of the last pooling layer
last_pooling_layer_index = next(i for i, layer in enumerate(all_layers) if isinstance(layer, nn.AdaptiveAvgPool2d))
print(f"Last Pooling Layer Index: {last_pooling_layer_index}")

/home/alan/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/alan/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer 0: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
Layer 1: BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Layer 2: ReLU(inplace=True)
Layer 3: MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Layer 4: Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d